# [How to do reduction in TVM](https://tvm.apache.org/docs/how_to/work_with_schedules/reduction.html#sphx-glr-how-to-work-with-schedules-reduction-py)

## SchedulePrimitives::Reduce

In [ ]:
from __future__ import absolute_import, print_function

import tvm
import tvm.testing
from tvm import te
import numpy as np

n = te.var("n")
m = te.var("m")

def test_reduce():
  A = te.placeholder((n, m), name="A")
  k = te.reduce_axis((0, m), "k")
  B = te.compute((n,), lambda i: te.sum(A[i, k], axis=k) , name="B")
  s = te.create_schedule(B.op)
  print(tvm.lower(s, [A, B], simple_mode=True))

test_reduce()

* Split reduce axis

In [ ]:
def test_split_reduce():
  A = te.placeholder((n, m), name="A")
  k = te.reduce_axis((0, m), "k")
  B = te.compute((n,), lambda i: te.sum(A[i, k], axis=k) , name="B")
  s = te.create_schedule(B.op)
  ko, ki = s[B].split(B.op.reduce_axis[0], factor=16)
  xo, xi = s[B].split(B.op.axis[0], factor=32)
  print(tvm.lower(s, [A, B], simple_mode=True))

test_split_reduce()

* Bind row in GPU kernel

In [ ]:
def test_reduce_bind():
  A = te.placeholder((n, m), name="A")
  k = te.reduce_axis((0, m), "k")
  B = te.compute((n,), lambda i: te.sum(A[i, k], axis=k) , name="B")
  s = te.create_schedule(B.op)
  ko, ki = s[B].split(B.op.reduce_axis[0], factor=16)
  xo, xi = s[B].split(B.op.axis[0], factor=32)
  s[B].bind(xo, te.thread_axis("blockIdx.x"))
  s[B].bind(xi, te.thread_axis("threadIdx.x"))
  print(tvm.lower(s, [A, B], simple_mode=True))

test_reduce_bind()

## SchedulePrimitives::Refactor

构建归约的一个问题是我们不能简单地在归约轴上并行化，需要划分归约的计算，将局部归约结果存储在临时数组中，然后再对临时数组进行归约。为了简化这个问题，引入 ___rfactor___ 原语对计算进行重写。下面这个这个列子对reduce维度进行了rfactor操作，目的是想让 _16_ 个 thread 能够同时进行规约计算，然后再使用 _1_ 个 thread 对 _16_ 个中间结果最后进行一次 _16_ 元素的规约，以便于GPU处理。

In [ ]:
def test_reduce_rfactor():
  A = te.placeholder((n, m), name="A")
  k = te.reduce_axis((0, m), "k")
  B = te.compute((n,), lambda i: te.sum(A[i, k], axis=k) , name="B")
  s = te.create_schedule(B.op)
  ko, ki = s[B].split(B.op.reduce_axis[0], factor=16)
  BF = s.rfactor(B, ki)
  print(tvm.lower(s, [A, B], simple_mode=True))
  print(s[B].op.body)

test_reduce_rfactor()

我们用 C 语言描述上面的计算过程:

```C++
#include <array>

// Row
#define N 100
// Col
#define M 256
// Parallel 
#define K 16

int main(void) {
  float A[N][M];
  float B[N];
  float B_rf[K][N];

  for(int k_inner = 0; k_inner < K; ++k_inner) {
    for (int i = 0; i < N; ++i) {
      B_rf[k_inner][i] = 0.0F;
      for(int k_outer = 0; k_outer < (M + 15) >> 4; ++k_outer) {
        if (k_outer * K + k_inner < M) {
          B_rf[k_inner][i] += A[i][k_outer * K + k_inner];
        }
      }
    }
  }
  for (int i = 0; i < N; ++i) {
    B[i] = 0.0F;
    for (int k = 0 ; k < K; ++k) {
      B[i] += B_rf[k];
    }
  }

  return 0;
}

```


## SchedulePrimitives::set_store_predicate

Cross Thread Reduction
这个例子展示了 ___rfactor___ 后的规约并行计算如何和 CUDA 编程模型进行 _bind_。这里我们在 ___set_store_predicate___ 的前后进行了结果对比，观察其行为。 ___set_store_predicate___ 设置或者不设置都不影响结果的正确性，它的语义是指定并行 axis 上的哪个 index (thread) 完成 store 的处理，如果没有指定的话那么16个thread会同时写结果到正确的位置上，不保证写的顺序。

In [ ]:
def cross_thread_reduction(show_cuda_code):
  A = te.placeholder((n, m), dtype='float32', name="A")
  k = te.reduce_axis((0, m), "k")
  B = te.compute((n,), lambda i: te.sum(A[i, k], axis=k) , name="B")
  s = te.create_schedule(B.op)
  ko, ki = s[B].split(B.op.reduce_axis[0], factor=16)
  BF = s.rfactor(B, ki)
  
  xo, xi = s[B].split(s[B].op.axis[0], factor=32)
  s[B].bind(xo, te.thread_axis("blockIdx.x"))
  s[B].bind(xi, te.thread_axis("threadIdx.y"))
  tx = te.thread_axis("threadIdx.x")
  s[B].bind(s[B].op.reduce_axis[0], tx)
  s[BF].compute_at(s[B], s[B].op.reduce_axis[0])
  s[B].set_store_predicate(tx.var.equal(0))
  print(tvm.lower(s, [A, B], simple_mode=True))
  fcuda = tvm.build(s, [A, B], "cuda")
  if show_cuda_code:
    print('*'*64)
    print('CUDA Source Code')
    print('*'*64)
    print(fcuda.imported_modules[0].get_source())
  return A, B, fcuda

results = cross_thread_reduction(show_cuda_code=False)

In [ ]:
def test_with_gpu(nn, A, B, fcuda):
  dev = tvm.cuda(0)
  a = tvm.nd.array(np.random.uniform(size=(nn, nn)).astype(A.dtype), dev)
  b = tvm.nd.array(np.zeros(nn, dtype=B.dtype), dev)
  fcuda(a, b)
  tvm.testing.assert_allclose(b.numpy(), np.sum(a.numpy(), axis=1), rtol=1e-6)

test_with_gpu(4096, results[0], results[1], results[2])

使用 ___reduce___ 实现一维卷积的例子：

In [ ]:
def test_conv1d():
  n = te.var("n")
  Input = te.placeholder((n, n), name="Input")
  Filter = te.placeholder((3, 3), name="Filter")
  di = te.reduce_axis((0, 3), name="di")
  dj = te.reduce_axis((0, 3), name="dj")
  Output = te.compute(
      (n - 2, n - 2),
      lambda i, j: te.sum(Input[i + di, j + dj] * Filter[di, dj], axis=[di, dj]),
      name="Output",
  )
  s = te.create_schedule(Output.op)
  print(tvm.lower(s, [Input, Filter, Output], simple_mode=True))

test_conv1d()

## SchedulePrimitives::comm_reducer

除了使用 tvm 内置的规约操作例如 te.sum, te.min, te.max 以外，也可以通过 ___reducer___ 自定义规约操作。

In [ ]:
def test_reducer():
  n = te.var("n")
  m = te.var("m")
  product = te.comm_reducer(lambda x, y: x * y, lambda t: tvm.tir.const(1, dtype=t), name="product")
  A = te.placeholder((n, m), name="A")
  k = te.reduce_axis((0, m), name="k")
  B = te.compute((n,), lambda i: product(A[i, k], axis=k), name="B")
  s = te.create_schedule(B.op)
  print(tvm.lower(s, [A, B], simple_mode=True))

test_reducer()